In [6]:
import pandas as pd

# Definimos la ruta a nuestro archivo de datos
data_path = "../data/raw/Tweets.csv"

# Cargamos los datos en un DataFrame de pandas
df = pd.read_csv(data_path)

# ¡Mostramos las primeras 5 filas para ver cómo se ven los datos!
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [7]:
# --- 1. Ver la distribución de los datos ---
# Queremos saber si los datos están balanceados (equilibrados)
print("Distribución de sentimientos:")
print(df['airline_sentiment'].value_counts())
print("\n" + "="*30 + "\n")


# --- 2. Crear un DataFrame limpio ---
# Seleccionamos solo las columnas que realmente usaremos
df_limpio = df[['text', 'airline_sentiment']]

# Mostramos las primeras 5 filas del nuevo DataFrame limpio
print("Datos limpios:")
df_limpio.head()

Distribución de sentimientos:
airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64


Datos limpios:


,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [8]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

# --- 1. Codificar las Etiquetas (Labels) ---

# Usaremos un LabelEncoder para convertir el texto a números
# negative -> 0, neutral -> 1, positive -> 2
encoder = LabelEncoder()
df_limpio['sentiment_encoded'] = encoder.fit_transform(df_limpio['airline_sentiment'])

# Vemos el resultado
print("Etiquetas codificadas:")
print(df_limpio[['airline_sentiment', 'sentiment_encoded']].head())
print("\n" + "="*30 + "\n")


# --- 2. Limpieza de Texto (Text Cleaning) ---

# Descargamos la lista de "stopwords" (palabras comunes como 'y', 'el', 'la', 'a')
# Esto solo se hace una vez
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Quitar @menciones
    texto = re.sub(r'@\w+', '', texto)
    # Quitar puntuación y números (dejar solo letras)
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)
    # Quitar stopwords
    texto = ' '.join(palabra for palabra in texto.split() if palabra not in stop_words)
    return texto

# Aplicamos la función de limpieza a toda la columna 'text'
print("Limpiando el texto...")
df_limpio['text_limpio'] = df_limpio['text'].apply(limpiar_texto)

# Mostramos el resultado final
print("Datos finales listos para el modelo:")
df_limpio[['text', 'text_limpio', 'sentiment_encoded']].head()

/tmp/ipykernel_25106/2184892709.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['sentiment_encoded'] = encoder.fit_transform(df_limpio['airline_sentiment'])
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/tmp/ipykernel_25106/2184892709.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_limpio['text_limpio'] = df_limpio['text'].apply(limpiar_texto)


Etiquetas codificadas:
  airline_sentiment  sentiment_encoded
0           neutral                  1
1          positive                  2
2           neutral                  1
3          negative                  0
4          negative                  0


Limpiando el texto...
Datos finales listos para el modelo:


,text,text_limpio,sentiment_encoded
0,@VirginAmerica What @dhepburn said.,said,1
1,@VirginAmerica plus you've added commercials t...,plus youve added commercials experience tacky,2
2,@VirginAmerica I didn't today... Must mean I n...,didnt today must mean need take another trip,1
3,@VirginAmerica it's really aggressive to blast...,really aggressive blast obnoxious entertainmen...,0
4,@VirginAmerica and it's a really big bad thing...,really big bad thing,0


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

# --- 1. Definir Parámetros ---
MAX_VOCAB_SIZE = 10000  # Cuántas palabras únicas recordará (las 10,000 más comunes)
MAX_SEQ_LENGTH = 50     # El largo máximo de un tuit (en palabras)

# --- 2. Tokenización ---
# Inicializa el Tokenizer
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)

# "Aprende" el vocabulario de nuestros textos limpios
tokenizer.fit_on_texts(df_limpio['text_limpio'])

# Convierte los textos en secuencias de números
X = tokenizer.texts_to_sequences(df_limpio['text_limpio'])

# --- 3. Padding (Relleno) ---
# Rellena (o corta) las secuencias para que TODAS midan MAX_SEQ_LENGTH
X_pad = pad_sequences(X, maxlen=MAX_SEQ_LENGTH, padding='post', truncating='post')

print(f"Forma de los datos de X (secuencias): {X_pad.shape}")
print("Ejemplo de secuencia rellenada:")
print(X_pad[1]) # Muestra el segundo tuit ("plus youve added...") en números

print("\n" + "="*30 + "\n")

# --- 4. Preparar las Etiquetas (Y) ---
# Necesitamos convertir nuestras etiquetas (0, 1, 2) al formato "one-hot"
# (0 -> [1,0,0], 1 -> [0,1,0], 2 -> [0,0,1])
y = to_categorical(df_limpio['sentiment_encoded'])

print(f"Forma de los datos de Y (etiquetas): {y.shape}")
print("Ejemplo de etiqueta 'one-hot' (era 'positive' -> 2):")
print(y[1]) # Muestra la etiqueta del segundo tuit

print("\n" + "="*30 + "\n")

# --- 5. División Train/Test ---
# Dividimos el 80% para entrenar y 20% para probar
X_train, X_test, y_train, y_test = train_test_split(
    X_pad, 
    y, 
    test_size=0.2, 
    random_state=42,  # Para que la división sea siempre igual
    stratify=y        # Asegura que tengamos % similares de 0, 1 y 2 en ambos sets
)

print("Datos listos:")
print(f"X_train (entrenamiento): {X_train.shape}")
print(f"y_train (entrenamiento): {y_train.shape}")
print(f"X_test (prueba): {X_test.shape}")
print(f"y_test (prueba): {y_test.shape}")

2025-11-17 01:29:16.314567: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 01:29:17.895600: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-17 01:29:20.506367: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Forma de los datos de X (secuencias): (14640, 50)
Ejemplo de secuencia rellenada:
[ 418  419  985 2219  112 5676    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


Forma de los datos de Y (etiquetas): (14640, 3)
Ejemplo de etiqueta 'one-hot' (era 'positive' -> 2):
[0. 0. 1.]


Datos listos:
X_train (entrenamiento): (11712, 50)
y_train (entrenamiento): (11712, 3)
X_test (prueba): (2928, 50)
y_test (prueba): (2928, 3)


In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# --- 1. Definir la Arquitectura ---

# Definimos el tamaño de entrada (nuestro MAX_SEQ_LENGTH)
input_layer = Input(shape=(MAX_SEQ_LENGTH,))

# Capa de Embedding
# (MAX_VOCAB_SIZE viene de la celda anterior, eran 10000)
# output_dim=128 significa que cada palabra será un vector de 128 dimensiones
embedding_layer = Embedding(input_dim=MAX_VOCAB_SIZE, 
                            output_dim=128, 
                            input_length=MAX_SEQ_LENGTH)(input_layer)

# Capa Bidireccional LSTM con 64 unidades de memoria
# Dropout=0.2 "apaga" el 20% de las neuronas al azar para evitar sobreajuste
lstm_layer = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(embedding_layer)

# Capa de salida
# 3 neuronas (una para cada clase) con activación 'softmax' (para probabilidades)
output_layer = Dense(3, activation='softmax')(lstm_layer)

# --- 2. Construir el Modelo ---
model = Model(inputs=input_layer, outputs=output_layer)

# --- 3. Compilar el Modelo ---
# 'adam' es el optimizador estándar
# 'categorical_crossentropy' es la función de pérdida para 3+ clases
# 'accuracy' es lo que queremos medir
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# --- 4. Ver el Resumen ---
# ¡Esto nos muestra un mapa de nuestro modelo!
model.summary()

/workspaces/sentiment-analysis-api/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-11-17 01:36:48.273361: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 50, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,379,203 (5.26 MB)

 Trainable params: 1,379,203 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# --- 1. Definir "Early Stopping" ---
# Esto es un guardián inteligente.
# Le decimos que "vigile" la pérdida en los datos de prueba (val_loss).
# Si el 'val_loss' no mejora durante 2 épocas seguidas,
# detendrá el entrenamiento automáticamente.
# Esto nos ahorra tiempo y evita que el modelo "sobreaprenda".
early_stop = EarlyStopping(monitor='val_loss', 
                             patience=2, 
                             restore_best_weights=True)

# --- 2. ¡Comenzar el Entrenamiento! ---
print("Comenzando el entrenamiento... ☕")

# Aquí le pasamos los datos de entrenamiento (X_train, y_train)
# Le decimos que use el 10% de esos datos para validación (validation_split=0.1)
# Le pedimos que entrene por un máximo de 10 épocas (epochs=10)
# Le decimos que use nuestro guardián (callbacks=[early_stop])
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.1,  # Usa el 10% para validar en cada época
    callbacks=[early_stop]
)

print("¡Entrenamiento finalizado! 🎉")

Comenzando el entrenamiento... ☕
Epoch 1/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 20s 97ms/step - accuracy: 0.6914 - loss: 0.7345 - val_accuracy: 0.7713 - val_loss: 0.5506
Epoch 2/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 16s 95ms/step - accuracy: 0.8294 - loss: 0.4429 - val_accuracy: 0.7841 - val_loss: 0.5217
Epoch 3/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 15s 93ms/step - accuracy: 0.8824 - loss: 0.3160 - val_accuracy: 0.7850 - val_loss: 0.5644
Epoch 4/10
165/165 ━━━━━━━━━━━━━━━━━━━━ 21s 97ms/step - accuracy: 0.9128 - loss: 0.2382 - val_accuracy: 0.7756 - val_loss: 0.6574
¡Entrenamiento finalizado! 🎉


In [12]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# --- 1. Evaluar el modelo en el Test Set ---
# Esto nos da la pérdida (loss) y la precisión (accuracy) finales
print("Evaluando el modelo en el set de prueba...")
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida en el Test Set: {loss:.4f}")
print(f"Precisión en el Test Set: {accuracy * 100:.2f}%")

print("\n" + "="*30 + "\n")

# --- 2. Generar Reporte de Clasificación ---
# Hacemos predicciones en el set de prueba
y_pred_probs = model.predict(X_test)
# Convertimos las probabilidades (ej. [0.1, 0.8, 0.1]) a una sola clase (ej. 1)
y_pred = np.argmax(y_pred_probs, axis=1)
# También necesitamos convertir el y_test (que es one-hot) a clases
y_test_labels = np.argmax(y_test, axis=1)

# Obtenemos los nombres de las etiquetas (ej. 0 -> 'negative')
# ¡OJO! El orden debe ser el que aprendió el encoder.
# Si 0=negative, 1=neutral, 2=positive...
target_names = encoder.classes_

# ¡Imprimimos el reporte!
print("Reporte de Clasificación:")
print(classification_report(y_test_labels, y_pred, target_names=target_names))

Evaluando el modelo en el set de prueba...
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7954 - loss: 0.5235
Pérdida en el Test Set: 0.5235
Precisión en el Test Set: 79.54%


92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step
Reporte de Clasificación:
              precision    recall  f1-score   support

    negative       0.83      0.92      0.88      1835
     neutral       0.66      0.51      0.57       620
    positive       0.76      0.69      0.72       473

    accuracy                           0.80      2928
   macro avg       0.75      0.71      0.72      2928
weighted avg       0.79      0.80      0.79      2928



In [13]:
import json

# --- 1. Definir las rutas donde guardaremos ---
model_path = "../src/model/sentiment_model.keras"
tokenizer_path = "../src/model/tokenizer.json"

# --- 2. Guardar el Modelo de Keras ---
print(f"Guardando el modelo en {model_path}...")
model.save(model_path)

# --- 3. Guardar el Tokenizer ---
# El tokenizer no se guarda con model.save(), hay que hacerlo aparte
print(f"Guardando el tokenizer en {tokenizer_path}...")
tokenizer_config = tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_config, ensure_ascii=False))

print("¡Modelo y tokenizer guardados con éxito! 💾")

Guardando el modelo en ../src/model/sentiment_model.keras...
Guardando el tokenizer en ../src/model/tokenizer.json...
¡Modelo y tokenizer guardados con éxito! 💾


In [14]:
import json

# --- 1. Definir las rutas donde guardaremos ---
model_path = "../src/model/sentiment_model.keras"
tokenizer_path = "../src/model/tokenizer.json"

# --- 2. Guardar el Modelo de Keras ---
print(f"Guardando el modelo en {model_path}...")
model.save(model_path)

# --- 3. Guardar el Tokenizer ---
# El tokenizer no se guarda con model.save(), hay que hacerlo aparte
print(f"Guardando el tokenizer en {tokenizer_path}...")
tokenizer_config = tokenizer.to_json()
with open(tokenizer_path, 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_config, ensure_ascii=False))

print("¡Modelo y tokenizer guardados con éxito! 💾")

Guardando el modelo en ../src/model/sentiment_model.keras...
Guardando el tokenizer en ../src/model/tokenizer.json...
¡Modelo y tokenizer guardados con éxito! 💾
